In [80]:
def detect_coins(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)

    circles = cv2.HoughCircles(
        gray,
        cv2.HOUGH_GRADIENT,
        dp=1.2,
        minDist=50,
        param1=50,
        param2=30,
        minRadius=50,
        maxRadius=100
    )

    annotated = image.copy()
    total = 0
    counts = {1:0, 5:0, 10:0, 20:0}

    if circles is not None:
        circles = np.uint16(np.around(circles[0, :]))
        detected = []

        for (x, y, r) in circles:
            if any(np.hypot(x - dx, y - dy) < 0.8 * r for dx, dy, dr in detected):
                continue
            detected.append((x, y, r))

            x1, y1 = max(0, x-r), max(0, y-r)
            x2, y2 = min(image.shape[1], x+r), min(image.shape[0], y+r)
            coin_roi = image[y1:y2, x1:x2]

            val = None
            hsv = cv2.cvtColor(coin_roi, cv2.COLOR_BGR2HSV)
            lower_gold = np.array([15, 50, 50])
            upper_gold = np.array([35, 255, 255])
            mask_gold = cv2.inRange(hsv, lower_gold, upper_gold)
            gold_ratio = np.sum(mask_gold > 0) / (coin_roi.shape[0] * coin_roi.shape[1])

            if gold_ratio > 0.03:
                val = 20
            else:
                if r < 40:
                    val = 1
                elif r < 60:
                    val = 5
                else:
                    val = 10

            counts[val] += 1
            total += val
            cv2.circle(annotated, (x, y), r, (0, 255, 0), 2)
            cv2.putText(annotated, f"PHP {val}", (x-20, y), cv2.FONT_HERSHEY_TRIPLEX, 0.6, (0,0,255), 2)

    cv2.putText(annotated, f"Total: PHP {total}", (20, 40), cv2.FONT_HERSHEY_TRIPLEX, 1, (255,255,255), 2)
    return annotated, counts, total


In [87]:
def detect_coins(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (7, 7), 1.5)

    circles = cv2.HoughCircles(
        gray, 
        cv2.HOUGH_GRADIENT, 
        dp=1.2, 
        minDist=50, 
        param1=100, 
        param2=50, 
        minRadius=30, 
        maxRadius=120
    )

    annotated = image.copy()
    total = 0
    counts = {1:0, 5:0, 10:0, 20:0}

    if circles is not None:
        circles = np.uint16(np.around(circles[0, :]))
        circles = sorted(circles, key=lambda c: c[2], reverse=True)
        detected = []

        for (x, y, r) in circles:
            if any(np.hypot(x - dx, y - dy) < 0.8 * r for dx, dy, dr in detected):
                continue
            detected.append((x, y, r))

            x1, y1 = max(0, x-r), max(0, y-r)
            x2, y2 = min(image.shape[1], x+r), min(image.shape[0], y+r)
            if x1 >= x2 or y1 >= y2:
                continue

            coin_roi = image[y1:y2, x1:x2]

            val = None
            if coin_roi.size == 0:
                continue

            hsv = cv2.cvtColor(coin_roi, cv2.COLOR_BGR2HSV)
            lower_gold = np.array([15, 80, 80])
            upper_gold = np.array([35, 255, 255])
            mask_gold = cv2.inRange(hsv, lower_gold, upper_gold)
            gold_ratio = np.sum(mask_gold > 0) / (coin_roi.shape[0] * coin_roi.shape[1])

            if gold_ratio > 0.05:
                val = 20
            else:
                if r < 55:
                    val = 1
                elif r < 70:
                    val = 5
                else:
                    val = 10

            counts[val] += 1
            total += val

            cv2.circle(annotated, (x, y), r, (0, 255, 0), 3)
            cv2.putText(annotated, f"PHP {val}", (x-30, y), cv2.FONT_HERSHEY_TRIPLEX, 0.8, (0, 0, 255), 2)

    cv2.putText(annotated, f"Total: PHP {total}", (20, 40), cv2.FONT_HERSHEY_TRIPLEX, 1.2, (255, 255, 255), 3)
    return annotated, counts, total


In [4]:
import tkinter as tk
import cv2
from PIL import Image, ImageTk
import numpy as np

class Activity3:
    def __init__(self, root):
        self.root = root
        self.root.title("Philippine Peso Coin Detector")

        self.video_label = tk.Label(root)
        self.video_label.pack()

        self.start_btn = tk.Button(root, text="Start Camera", command=self.start_camera)
        self.start_btn.pack(side=tk.LEFT, padx=5, pady=5)

        self.stop_btn = tk.Button(root, text="Stop Camera", command=self.stop_camera)
        self.stop_btn.pack(side=tk.LEFT, padx=5, pady=5)

        self.snapshot_btn = tk.Button(root, text="Save Snapshot", command=self.save_snapshot)
        self.snapshot_btn.pack(side=tk.LEFT, padx=5, pady=5)

        self.cap = None
        self.frame = None
        self.running = False 

    def start_camera(self):
        if not self.running:
            self.cap = cv2.VideoCapture(0)
            self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
            self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
            self.cap.set(cv2.CAP_PROP_FPS, 30)
            self.running = True
            self.update_frame()

    def stop_camera(self):
        if self.running:
            self.running = False
            if self.cap:
                self.cap.release()

    def save_snapshot(self):
        if self.frame is not None:
            cv2.imwrite("snapshot.png", cv2.cvtColor(self.frame, cv2.COLOR_BGR2RGB))
            print("Snapshot saved as snapshot.png")

    def update_frame(self):
        if self.running:
            ret, frame = self.cap.read()
            if ret:
                frame, counts, total = detect_coins(frame)
                self.frame = frame
                img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                img = Image.fromarray(img)
                imgtk = ImageTk.PhotoImage(image=img)
                self.video_label.imgtk = imgtk
                self.video_label.configure(image=imgtk)
            self.root.after(10, self.update_frame)


In [6]:
root = tk.Tk()
app = Activity3(root)
root.mainloop()